In [1]:
from ollama_qw import OllamaQW
qw = OllamaQW()

2025-01-16 09:32:24.781 | INFO     | ollama_qw:__init__:12 - [{'type': 'function', 'function': {'name': 'close_light', 'description': '关掉机箱的灯。', 'parameters': {'type': 'object', 'properties': {}, 'required': []}}}, {'type': 'function', 'function': {'name': 'open_light', 'description': '打开机箱的灯。', 'parameters': {'type': 'object', 'properties': {}, 'required': []}}}, {'type': 'function', 'function': {'name': 'run_software', 'description': '打开或者运行程序、软件、应用、脚本等。', 'parameters': {'type': 'object', 'properties': {'software_name': {'type': 'string', 'description': '需要打开或者运行的程序的简称或者别称，从enum中选择最相符的传入需要打开、运行或者执行的软件程序或者脚本的名称', 'enum': ['administrative tools', 'file explorer', 'ollama', 'sonic suite companion', 'wps office', 'livecaptions', 'magnify', 'narrator', 'on-screen keyboard', 'voiceaccess', 'blender 4', 'directx修复工具', 'gw-soul', 'unwise', 'kook', 'letsvpn', 'overwolf', 'valorant tracker', 'postman', 'raspberry pi imager', 'openrgb', 'sakuralauncher_8a49d589b8997c26e5fe8ca0c8308d23', 'steam'

In [2]:

def main():
    while True:
        msg = input(">: ")
        if msg.lower() == '/exit':
            break
        res = qw.chat(msg)
        print(f"Alice: {res['content']}")
main()

2025-01-16 09:32:33.858 | INFO     | ollama_qw:chat:27 - input message: {'role': 'user', 'content': '打开cpuz'}
2025-01-16 09:32:34.067 | INFO     | ollama_qw:ollama_chat:21 - model response is: model='qwen2.5:3b' created_at='2025-01-16T01:32:34.0660627Z' done=True done_reason='stop' total_duration=205175200 load_duration=11000900 prompt_eval_count=278 prompt_eval_duration=3000000 eval_count=23 eval_duration=188000000 message=Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='run_software', arguments={'software_name': 'cpuz'}))])
2025-01-16 09:32:34.067 | INFO     | ollama_qw:chat:35 - function name: run_software
2025-01-16 09:32:34.068 | INFO     | ollama_qw:chat:36 - function args: {'software_name': 'cpuz'}
2025-01-16 09:32:34.117 | INFO     | ollama_qw:chat:44 - tool response is: {'role': 'tool', 'name': 'run_software', 'content': '{"status": 200, "message": "\\u7a0b\\u5e8f\\u5df2\\u542f\\u52a8"}'}
2025-01-16 09:32:34.278 | INFO     | ollam

Alice: 成功启动了CPZ，你可以开始使用它来查看CPU信息了。


2025-01-16 09:33:00.313 | INFO     | ollama_qw:chat:27 - input message: {'role': 'user', 'content': '谢谢你'}
2025-01-16 09:33:00.424 | INFO     | ollama_qw:ollama_chat:21 - model response is: model='qwen2.5:3b' created_at='2025-01-16T01:33:00.4232529Z' done=True done_reason='stop' total_duration=108506200 load_duration=12001100 prompt_eval_count=381 prompt_eval_duration=4000000 eval_count=10 eval_duration=84000000 message=Message(role='assistant', content='不客气，请问接下来有什么可以帮助你的？', images=None, tool_calls=None)


Alice: 不客气，请问接下来有什么可以帮助你的？


2025-01-16 09:33:11.500 | INFO     | ollama_qw:chat:27 - input message: {'role': 'user', 'content': ''}
2025-01-16 09:33:11.718 | INFO     | ollama_qw:ollama_chat:21 - model response is: model='qwen2.5:3b' created_at='2025-01-16T01:33:11.7173688Z' done=True done_reason='stop' total_duration=215066900 load_duration=10499100 prompt_eval_count=400 prompt_eval_duration=8000000 eval_count=23 eval_duration=185000000 message=Message(role='assistant', content='目前没有其他帮助的请求了。如果你需要更多关于CPZ的信息或者其他的帮助，请告诉我！\n', images=None, tool_calls=None)


Alice: 目前没有其他帮助的请求了。如果你需要更多关于CPZ的信息或者其他的帮助，请告诉我！



2025-01-16 09:33:16.628 | INFO     | ollama_qw:chat:27 - input message: {'role': 'user', 'content': 'exit'}
2025-01-16 09:33:16.867 | INFO     | ollama_qw:ollama_chat:21 - model response is: model='qwen2.5:3b' created_at='2025-01-16T01:33:16.8668564Z' done=True done_reason='stop' total_duration=236999700 load_duration=13002900 prompt_eval_count=433 prompt_eval_duration=4000000 eval_count=25 eval_duration=205000000 message=Message(role='assistant', content='已经为您准备好了，如果您有任何问题或需要进一步的帮助，请随时联系我。祝您有一个愉快的一天！', images=None, tool_calls=None)


Alice: 已经为您准备好了，如果您有任何问题或需要进一步的帮助，请随时联系我。祝您有一个愉快的一天！


In [3]:
import sys
from PyQt5.QtCore import Qt
from PyQt5.QtWidgets import QApplication, QMainWindow, QTextEdit, QLineEdit, QVBoxLayout, QWidget, QSizePolicy


class TransparentTerminal(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("自定义终端")
        self.setGeometry(100, 100, 600, 400)

        # 设置主窗口
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)

        # 创建文本框，显示终端内容
        self.text_display = QTextEdit(self)
        self.text_display.setReadOnly(True)
        self.text_display.setStyleSheet("background-color: transparent; color: black;")

        # 创建输入框
        self.input_box = QLineEdit(self)
        self.input_box.setPlaceholderText("输入命令...")
        self.input_box.setStyleSheet("background-color: transparent; color: black;")

        # 创建布局并添加控件
        self.layout = QVBoxLayout(self.central_widget)
        self.layout.addWidget(self.text_display)
        self.layout.addWidget(self.input_box)

        # 设置输入框大小策略
        self.input_box.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Fixed)

        # 连接输入框事件
        self.input_box.returnPressed.connect(self.handle_input)

    def handle_input(self):
        command = self.input_box.text().strip()
        self.text_display.append(f"> {command}")  # 在文本框中显示输入的命令
        self.input_box.clear()

        # 处理命令
        if command == "exit":
            self.text_display.append("退出终端...")
            self.close()
        elif command == "hello":
            self.text_display.append("你好，欢迎来到自定义终端！")
        else:
            self.text_display.append(f"未知命令: {command}")


def main():
    app = QApplication(sys.argv)
    terminal = TransparentTerminal()
    terminal.show()
    app.exec_()



main()


In [1]:
import pandas as pd

fixed_replay_path = r'data/fixed_replay.xlsx'
fixed_replay = {}
df = pd.read_excel(fixed_replay_path)
for index,row in df.iterrows():
    if row['key'] in fixed_replay:
        fixed_replay[row['key']].append(row['response'])
    else:
        fixed_replay[row['key']] = []
        fixed_replay[row['key']].append(row['response'])

In [5]:
len(list(fixed_replay.keys()))

12210

In [10]:
import difflib

difflib.get_close_matches('你你你你你',fixed_replay.keys(),2, cutoff=0.8)

TypeError: object of type 'float' has no len()

In [3]:
from vectordb import Memory

memory = Memory()

# text = "..."
# metadata = {...}
texts = ["这是一个苹果的例子", "橙子通常是橙色的"]
metadatas = [[{"source": "水果知识一"}], [{"source": "水果知识二"}]]
# Save text with metadata
# This will automatically embed content
memory.save(texts, metadatas)
query = "苹果"
# Search for top n relevant chunks
# We will automatically use the fastest vector search backend
results = memory.search(query, top_n=3)

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [2]:
from vectordb import Memory
# 初始化Memory对象作为存储库
memory = Memory()
# 示例数据: 文本内容及其关联元数据
texts = ["这是一个苹果的例子", "橙子通常是橙色的"]
metadatas = [[{"source": "水果知识一"}], [{"source": "水果知识二"}]]
# 保存文本和元数据到Memory中，自动完成内容的嵌入
for text, meta in zip(texts, metadatas):
    memory.save(text, meta)
# 使用关键词查询，自动找到最相关的片段
query = "苹果"
results = memory.search(query, top_n=1)
print("查询结果:", results)

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle